In [21]:
# %matplotlib inline

In [22]:
# !pip install dgl==0.6.1


Link Prediction using Graph Neural Networks
===========================================
how to train a GNN for link prediction, i.e. predicting the
existence of an edge between two arbitrary nodes in a graph.




In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp

ModuleNotFoundError: No module named 'dgl'

Overview of Link Prediction with GNN
------------------------------------

Many applications such as social recommendation, item recommendation,
knowledge graph completion, etc., can be formulated as link prediction,
which predicts whether an edge exists between two particular nodes. This
tutorial shows an example of predicting whether a citation relationship,
either citing or being cited, between two papers exists in a citation
network.

This tutorial formulates the link prediction problem as a binary classification
problem as follows:

-  Treat the edges in the graph as *positive examples*.
-  Sample a number of non-existent edges (i.e. node pairs with no edges
   between them) as *negative* examples.
-  Divide the positive examples and negative examples into a training
   set and a test set.
-  Evaluate the model with any binary classification metric such as Area
   Under Curve (AUC).

<div class="alert alert-info"><h4>Note</h4><p>The practice comes from
   `SEAL <https://papers.nips.cc/paper/2018/file/53f0d7c537d99b3824f0f99d62ea2428-Paper.pdf>`__,
   although the model here does not use their idea of node labeling.</p></div>

In some domains such as large-scale recommender systems or information
retrieval, you may favor metrics that emphasize good performance of
top-K predictions. In these cases you may want to consider other metrics
such as mean average precision, and use other negative sampling methods,
which are beyond the scope of this tutorial.

Loading graph and features
--------------------------






In [2]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


Prepare training and testing sets
---------------------------------

This randomly picks 10% of the edges for positive examples in
the test set, and leave the rest for the training set. It then samples
the same number of edges for negative examples in both sets.




In [3]:
# Split edge set for training and testing
u, v = g.edges()

print("node 1 list:",u)

print("node 2 list:", v)

eids = np.arange(g.number_of_edges())
print(eids)
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))

# print(adj)

adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())

print(adj_neg)

neg_u, neg_v = np.where(adj_neg != 0)

print(neg_u)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]


node 1 list: tensor([   0,    0,    0,  ..., 2707, 2707, 2707])
node 2 list: tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706])
[    0     1     2 ... 10553 10554 10555]
[[0. 1. 1. ... 1. 1. 1.]
 [1. 0. 0. ... 1. 1. 1.]
 [1. 0. 0. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 0. 0.]]
[   0    0    0 ... 2707 2707 2707]


When training, you will need to remove the edges in the test set from
the original graph. You can do this via ``dgl.remove_edges``.

<div class="alert alert-info"><h4>Note</h4><p>``dgl.remove_edges`` works by creating a subgraph from the
   original graph, resulting in a copy and therefore could be slow for
   large graphs. If so, you could save the training and test graph to
   disk, as you would do for preprocessing.</p></div>




In [4]:
train_g = dgl.remove_edges(g, eids[:test_size])

Define a GraphSAGE model
------------------------

This tutorial builds a model consisting of two
`GraphSAGE <https://arxiv.org/abs/1706.02216>`__ layers, each computes
new node representations by averaging neighbor information. DGL provides
``dgl.nn.SAGEConv`` that conveniently creates a GraphSAGE layer.




In [5]:
from dgl.nn import SAGEConv

#  2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

The model then predicts the probability of existence of an edge by
computing a score between the representations of both incident nodes
with a function (e.g. an MLP or a dot product), which you will see in
the next section.




Positive graph, negative graph, and ``apply_edges``
---------------------------------------------------

In previous tutorials you have learned how to compute node
representations with a GNN. However, link prediction requires you to
compute representation of *pairs of nodes*.

DGL recommends you to treat the pairs of nodes as another graph, since
you can describe a pair of nodes with an edge. In link prediction, you
will have a *positive graph* consisting of all the positive examples as
edges, and a *negative graph* consisting of all the negative examples.
The *positive graph* and the *negative graph* will contain the same set
of nodes as the original graph.  This makes it easier to pass node
features among multiple graphs for computation.  As you will see later,
you can directly feed the node representations computed on the entire
graph to the positive and the negative graphs for computing pair-wise
scores.

The following code constructs the positive graph and the negative graph
for the training set and the test set respectively.




In [6]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

The benefit of treating the pairs of nodes as a graph is that you can
use the ``DGLGraph.apply_edges`` method, which conveniently computes new
edge features based on the incident nodes’ features and the original
edge features (if applicable).

DGL provides a set of optimized builtin functions to compute new
edge features based on the original node/edge features. For example,
``dgl.function.u_dot_v`` computes a dot product of the incident nodes’
representations for each edge.




In [7]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

You can also write your own function if it is complex.
For instance, the following module produces a scalar score on each edge
by concatenating the incident nodes’ features and passing it to an MLP.




In [8]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

Training loop
-------------




In [9]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16)
pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [10]:
# 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

In [13]:
#  4. training -------------------------------- #
all_logits = []
for e in range(15000):
    # forward
    h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print('In epoch {}, loss: {}'.format(e, loss))
    
#     if e % 5 == 0:
#         print('In epoch {}, loss: {}'.format(e, loss))

In epoch 0, loss: 0.015991540625691414
In epoch 1, loss: 0.01522025279700756
In epoch 2, loss: 0.014488667249679565
In epoch 3, loss: 0.013795426115393639
In epoch 4, loss: 0.01313852146267891
In epoch 5, loss: 0.012517143972218037
In epoch 6, loss: 0.011928597465157509
In epoch 7, loss: 0.011371511034667492
In epoch 8, loss: 0.010844390839338303
In epoch 9, loss: 0.01034609042108059
In epoch 10, loss: 0.009874791838228703
In epoch 11, loss: 0.009429256431758404
In epoch 12, loss: 0.009008048102259636
In epoch 13, loss: 0.00860985554754734
In epoch 14, loss: 0.008233529515564442
In epoch 15, loss: 0.00787776242941618
In epoch 16, loss: 0.007541427854448557
In epoch 17, loss: 0.007223689928650856
In epoch 18, loss: 0.006923088803887367
In epoch 19, loss: 0.006638649385422468
In epoch 20, loss: 0.0063698263838887215
In epoch 21, loss: 0.006115003488957882
In epoch 22, loss: 0.005874175578355789
In epoch 23, loss: 0.00564629677683115
In epoch 24, loss: 0.005430480930954218
In epoch 25, lo

In epoch 204, loss: 0.00033884093863889575
In epoch 205, loss: 0.000336184399202466
In epoch 206, loss: 0.00033356473431922495
In epoch 207, loss: 0.0003309807798359543
In epoch 208, loss: 0.00032843247754499316
In epoch 209, loss: 0.00032591953640803695
In epoch 210, loss: 0.00032343933708034456
In epoch 211, loss: 0.0003209922288078815
In epoch 212, loss: 0.0003185776877216995
In epoch 213, loss: 0.0003161923086736351
In epoch 214, loss: 0.0003138399915769696
In epoch 215, loss: 0.0003115180297754705
In epoch 216, loss: 0.00030922528821974993
In epoch 217, loss: 0.0003069609811063856
In epoch 218, loss: 0.0003047254285775125
In epoch 219, loss: 0.00030251662246882915
In epoch 220, loss: 0.0003003353194799274
In epoch 221, loss: 0.0002981805009767413
In epoch 222, loss: 0.00029605207964777946
In epoch 223, loss: 0.00029394871671684086
In epoch 224, loss: 0.0002918708196375519
In epoch 225, loss: 0.00028981821378692985
In epoch 226, loss: 0.00028778842533938587
In epoch 227, loss: 0.00

In epoch 420, loss: 0.00010672707139747217
In epoch 421, loss: 0.00010630912584019825
In epoch 422, loss: 0.00010589403245830908
In epoch 423, loss: 0.00010548155114520341
In epoch 424, loss: 0.00010507191473152488
In epoch 425, loss: 0.00010466433741385117
In epoch 426, loss: 0.00010425884102005512
In epoch 427, loss: 0.00010385583300376311
In epoch 428, loss: 0.00010345552436774597
In epoch 429, loss: 0.00010305763134965673
In epoch 430, loss: 0.00010266165190842003
In epoch 431, loss: 0.00010226828453596681
In epoch 432, loss: 0.00010187731095356867
In epoch 433, loss: 0.00010148860019398853
In epoch 434, loss: 0.00010110175207955763
In epoch 435, loss: 0.00010071777796838433
In epoch 436, loss: 0.00010033622675109655
In epoch 437, loss: 9.99562835204415e-05
In epoch 438, loss: 9.957821748685092e-05
In epoch 439, loss: 9.920298180077225e-05
In epoch 440, loss: 9.88297033472918e-05
In epoch 441, loss: 9.845844033407047e-05
In epoch 442, loss: 9.808954928303137e-05
In epoch 443, loss:

In epoch 626, loss: 5.440514360088855e-05
In epoch 627, loss: 5.425487688626163e-05
In epoch 628, loss: 5.410531957750209e-05
In epoch 629, loss: 5.395651169237681e-05
In epoch 630, loss: 5.380807488108985e-05
In epoch 631, loss: 5.366064215195365e-05
In epoch 632, loss: 5.3513133025262505e-05
In epoch 633, loss: 5.3366689826361835e-05
In epoch 634, loss: 5.3221108828438446e-05
In epoch 635, loss: 5.307522224029526e-05
In epoch 636, loss: 5.293090725899674e-05
In epoch 637, loss: 5.2786472224397585e-05
In epoch 638, loss: 5.264352512313053e-05
In epoch 639, loss: 5.2500352467177436e-05
In epoch 640, loss: 5.23579292348586e-05
In epoch 641, loss: 5.2215786126907915e-05
In epoch 642, loss: 5.2074847189942375e-05
In epoch 643, loss: 5.1934308430645615e-05
In epoch 644, loss: 5.1794020691886544e-05
In epoch 645, loss: 5.16548752784729e-05
In epoch 646, loss: 5.15158535563387e-05
In epoch 647, loss: 5.137767584528774e-05
In epoch 648, loss: 5.123965092934668e-05
In epoch 649, loss: 5.110241

In epoch 832, loss: 3.292133260401897e-05
In epoch 833, loss: 3.284995545982383e-05
In epoch 834, loss: 3.2779502362245694e-05
In epoch 835, loss: 3.270861634518951e-05
In epoch 836, loss: 3.263790495111607e-05
In epoch 837, loss: 3.2567786547588184e-05
In epoch 838, loss: 3.2497617212357e-05
In epoch 839, loss: 3.2427607948193327e-05
In epoch 840, loss: 3.235810072510503e-05
In epoch 841, loss: 3.2289084629155695e-05
In epoch 842, loss: 3.222001396352425e-05
In epoch 843, loss: 3.2150761398952454e-05
In epoch 844, loss: 3.208213820471428e-05
In epoch 845, loss: 3.2013434974942356e-05
In epoch 846, loss: 3.1944786314852536e-05
In epoch 847, loss: 3.187709444318898e-05
In epoch 848, loss: 3.180916974088177e-05
In epoch 849, loss: 3.174148878315464e-05
In epoch 850, loss: 3.1673855119151995e-05
In epoch 851, loss: 3.160668711643666e-05
In epoch 852, loss: 3.1540072086500004e-05
In epoch 853, loss: 3.147281313431449e-05
In epoch 854, loss: 3.1406259950017557e-05
In epoch 855, loss: 3.1340

...

In epoch 14068, loss: 2.6348750958860023e-10
In epoch 14069, loss: 2.572140000989265e-10
In epoch 14070, loss: 2.383934716299052e-10
In epoch 14071, loss: 2.9485508479254463e-10
In epoch 14072, loss: 2.8858157530287087e-10
In epoch 14073, loss: 3.262226322409134e-10
In epoch 14074, loss: 2.9485508479254463e-10
In epoch 14075, loss: 3.011285942822184e-10
In epoch 14076, loss: 2.823080380576215e-10
In epoch 14077, loss: 2.7603452856794775e-10
In epoch 14078, loss: 2.69761019078274e-10
In epoch 14079, loss: 2.509404906092527e-10
In epoch 14080, loss: 2.509404906092527e-10
In epoch 14081, loss: 3.136756132615659e-10
In epoch 14082, loss: 3.011285942822184e-10
In epoch 14083, loss: 2.69761019078274e-10
In epoch 14084, loss: 2.509404906092527e-10
In epoch 14085, loss: 2.823080380576215e-10
In epoch 14086, loss: 2.509404906092527e-10
In epoch 14087, loss: 2.823080380576215e-10
In epoch 14088, loss: 2.823080380576215e-10
In epoch 14089, loss: 3.136756132615659e-10
In epoch 14090, loss: 2.69761

In epoch 14267, loss: 2.3211994826244364e-10
In epoch 14268, loss: 1.505642971411092e-10
In epoch 14269, loss: 1.944788774466133e-10
In epoch 14270, loss: 1.505642971411092e-10
In epoch 14271, loss: 1.944788774466133e-10
In epoch 14272, loss: 2.509404906092527e-10
In epoch 14273, loss: 1.944788774466133e-10
In epoch 14274, loss: 1.6938483948791827e-10
In epoch 14275, loss: 2.3211994826244364e-10
In epoch 14276, loss: 2.070258964259608e-10
In epoch 14277, loss: 1.505642971411092e-10
In epoch 14278, loss: 1.6938483948791827e-10
In epoch 14279, loss: 1.944788774466133e-10
In epoch 14280, loss: 1.944788774466133e-10
In epoch 14281, loss: 1.8193185846726578e-10
In epoch 14282, loss: 2.1957292928309613e-10
In epoch 14283, loss: 1.505642971411092e-10
In epoch 14284, loss: 2.0075240081407486e-10
In epoch 14285, loss: 2.1329940591563457e-10
In epoch 14286, loss: 2.0075240081407486e-10
In epoch 14287, loss: 2.3211994826244364e-10
In epoch 14288, loss: 1.944788774466133e-10
In epoch 14289, loss: 

In epoch 14464, loss: 1.3801726428397387e-10
In epoch 14465, loss: 1.3801726428397387e-10
In epoch 14466, loss: 1.5683780663078295e-10
In epoch 14467, loss: 1.3801726428397387e-10
In epoch 14468, loss: 1.5683780663078295e-10
In epoch 14469, loss: 1.4429078765143544e-10
In epoch 14470, loss: 1.5683780663078295e-10
In epoch 14471, loss: 1.3801726428397387e-10
In epoch 14472, loss: 1.2547024530462636e-10
In epoch 14473, loss: 1.2547024530462636e-10
In epoch 14474, loss: 1.191967358149526e-10
In epoch 14475, loss: 2.0075240081407486e-10
In epoch 14476, loss: 1.4429078765143544e-10
In epoch 14477, loss: 1.3174375479430012e-10
In epoch 14478, loss: 1.3801726428397387e-10
In epoch 14479, loss: 1.2547024530462636e-10
In epoch 14480, loss: 1.3801726428397387e-10
In epoch 14481, loss: 1.8820536795693954e-10
In epoch 14482, loss: 1.1292321938638494e-10
In epoch 14483, loss: 1.631113161204567e-10
In epoch 14484, loss: 9.410268397846977e-11
In epoch 14485, loss: 1.4429078765143544e-10
In epoch 1448

In epoch 14662, loss: 1.3801726428397387e-10
In epoch 14663, loss: 1.1292321938638494e-10
In epoch 14664, loss: 1.1292321938638494e-10
In epoch 14665, loss: 1.3174375479430012e-10
In epoch 14666, loss: 1.1292321938638494e-10
In epoch 14667, loss: 1.191967358149526e-10
In epoch 14668, loss: 1.4429078765143544e-10
In epoch 14669, loss: 1.8820536795693954e-10
In epoch 14670, loss: 1.191967358149526e-10
In epoch 14671, loss: 1.2547024530462636e-10
In epoch 14672, loss: 1.2547024530462636e-10
In epoch 14673, loss: 9.410268397846977e-11
In epoch 14674, loss: 2.1329940591563457e-10
In epoch 14675, loss: 1.505642971411092e-10
In epoch 14676, loss: 1.1292321938638494e-10
In epoch 14677, loss: 1.2547024530462636e-10
In epoch 14678, loss: 1.0037620040703743e-10
In epoch 14679, loss: 9.410268397846977e-11
In epoch 14680, loss: 1.1292321938638494e-10
In epoch 14681, loss: 9.410268397846977e-11
In epoch 14682, loss: 1.2547024530462636e-10
In epoch 14683, loss: 1.1292321938638494e-10
In epoch 14684, 

In epoch 14863, loss: 1.3801726428397387e-10
In epoch 14864, loss: 1.3174375479430012e-10
In epoch 14865, loss: 1.1292321938638494e-10
In epoch 14866, loss: 9.410268397846977e-11
In epoch 14867, loss: 8.155565806022835e-11
In epoch 14868, loss: 7.52821485705546e-11
In epoch 14869, loss: 8.782917448879601e-11
In epoch 14870, loss: 1.0037619346814353e-10
In epoch 14871, loss: 8.782917448879601e-11
In epoch 14872, loss: 6.273512265231318e-11
In epoch 14873, loss: 1.1292321938638494e-10
In epoch 14874, loss: 1.0037620040703743e-10
In epoch 14875, loss: 9.410268397846977e-11
In epoch 14876, loss: 9.410268397846977e-11
In epoch 14877, loss: 1.0664970295781728e-10
In epoch 14878, loss: 8.155565806022835e-11
In epoch 14879, loss: 9.410268397846977e-11
In epoch 14880, loss: 1.191967358149526e-10
In epoch 14881, loss: 1.0037620040703743e-10
In epoch 14882, loss: 1.0664970295781728e-10
In epoch 14883, loss: 9.410268397846977e-11
In epoch 14884, loss: 8.782917448879601e-11
In epoch 14885, loss: 7.

In [14]:
# 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))


# Thumbnail credits: Link Prediction with Neo4j, Mark Needham
# sphinx_gallery_thumbnail_path = '_static/blitz_4_link_predict.png'

AUC 0.8354601199433974


In [15]:
import math
import os.path as osp
from itertools import chain

import numpy as np
import torch
import torch.nn.functional as F
from scipy.sparse.csgraph import shortest_path
from sklearn.metrics import roc_auc_score
from torch.nn import BCEWithLogitsLoss, Conv1d, MaxPool1d, ModuleList

from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, GCNConv, global_sort_pool
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import k_hop_subgraph, to_scipy_sparse_matrix


In [16]:
class SEALDataset(InMemoryDataset):
    def __init__(self, dataset, num_hops, split='train'):
        self.data = dataset[0]
        self.num_hops = num_hops
        super().__init__(dataset.root)
        index = ['train', 'val', 'test'].index(split)
        self.data, self.slices = torch.load(self.processed_paths[index])

    @property
    def processed_file_names(self):
        return ['SEAL_train_data.pt', 'SEAL_val_data.pt', 'SEAL_test_data.pt']

    def process(self):
        transform = RandomLinkSplit(num_val=0.05, num_test=0.1,
                                    is_undirected=True, split_labels=True)
        train_data, val_data, test_data = transform(self.data)

        self._max_z = 0

        # Collect a list of subgraphs for training, validation and testing:
        train_pos_data_list = self.extract_enclosing_subgraphs(
            train_data.edge_index, train_data.pos_edge_label_index, 1)
        train_neg_data_list = self.extract_enclosing_subgraphs(
            train_data.edge_index, train_data.neg_edge_label_index, 0)

        val_pos_data_list = self.extract_enclosing_subgraphs(
            val_data.edge_index, val_data.pos_edge_label_index, 1)
        val_neg_data_list = self.extract_enclosing_subgraphs(
            val_data.edge_index, val_data.neg_edge_label_index, 0)

        test_pos_data_list = self.extract_enclosing_subgraphs(
            test_data.edge_index, test_data.pos_edge_label_index, 1)
        test_neg_data_list = self.extract_enclosing_subgraphs(
            test_data.edge_index, test_data.neg_edge_label_index, 0)

        # Convert node labeling to one-hot features.
        for data in chain(train_pos_data_list, train_neg_data_list,
                          val_pos_data_list, val_neg_data_list,
                          test_pos_data_list, test_neg_data_list):
            # We solely learn links from structure, dropping any node features:
            data.x = F.one_hot(data.z, self._max_z + 1).to(torch.float)

        torch.save(self.collate(train_pos_data_list + train_neg_data_list),
                   self.processed_paths[0])
        torch.save(self.collate(val_pos_data_list + val_neg_data_list),
                   self.processed_paths[1])
        torch.save(self.collate(test_pos_data_list + test_neg_data_list),
                   self.processed_paths[2])

    def extract_enclosing_subgraphs(self, edge_index, edge_label_index, y):
        data_list = []
        for src, dst in edge_label_index.t().tolist():
            sub_nodes, sub_edge_index, mapping, _ = k_hop_subgraph(
                [src, dst], self.num_hops, edge_index, relabel_nodes=True)
            src, dst = mapping.tolist()

            # Remove target link from the subgraph.
            mask1 = (sub_edge_index[0] != src) | (sub_edge_index[1] != dst)
            mask2 = (sub_edge_index[0] != dst) | (sub_edge_index[1] != src)
            sub_edge_index = sub_edge_index[:, mask1 & mask2]

            # Calculate node labeling.
            z = self.drnl_node_labeling(sub_edge_index, src, dst,
                                        num_nodes=sub_nodes.size(0))

            data = Data(x=self.data.x[sub_nodes], z=z,
                        edge_index=sub_edge_index, y=y)
            data_list.append(data)

        return data_list

    def drnl_node_labeling(self, edge_index, src, dst, num_nodes=None):
        # Double-radius node labeling (DRNL).
        src, dst = (dst, src) if src > dst else (src, dst)
        adj = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes).tocsr()

        idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
        adj_wo_src = adj[idx, :][:, idx]

        idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
        adj_wo_dst = adj[idx, :][:, idx]

        dist2src = shortest_path(adj_wo_dst, directed=False, unweighted=True,
                                 indices=src)
        dist2src = np.insert(dist2src, dst, 0, axis=0)
        dist2src = torch.from_numpy(dist2src)

        dist2dst = shortest_path(adj_wo_src, directed=False, unweighted=True,
                                 indices=dst - 1)
        dist2dst = np.insert(dist2dst, src, 0, axis=0)
        dist2dst = torch.from_numpy(dist2dst)

        dist = dist2src + dist2dst
        dist_over_2, dist_mod_2 = dist // 2, dist % 2

        z = 1 + torch.min(dist2src, dist2dst)
        z += dist_over_2 * (dist_over_2 + dist_mod_2 - 1)
        z[src] = 1.
        z[dst] = 1.
        z[torch.isnan(z)] = 0.

        self._max_z = max(int(z.max()), self._max_z)

        return z.to(torch.long)


path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Planetoid')
dataset = Planetoid(path, name='Cora')

train_dataset = SEALDataset(dataset, num_hops=2, split='train')
val_dataset = SEALDataset(dataset, num_hops=2, split='val')
test_dataset = SEALDataset(dataset, num_hops=2, split='test')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


class DGCNN(torch.nn.Module):
    def __init__(self, hidden_channels, num_layers, GNN=GCNConv, k=0.6):
        super().__init__()

        if k < 1:  # Transform percentile to number.
            num_nodes = sorted([data.num_nodes for data in train_dataset])
            k = num_nodes[int(math.ceil(k * len(num_nodes))) - 1]
            k = max(10, k)
        self.k = int(k)

        self.convs = ModuleList()
        self.convs.append(GNN(train_dataset.num_features, hidden_channels))
        for i in range(0, num_layers - 1):
            self.convs.append(GNN(hidden_channels, hidden_channels))
        self.convs.append(GNN(hidden_channels, 1))

        conv1d_channels = [16, 32]
        total_latent_dim = hidden_channels * num_layers + 1
        conv1d_kws = [total_latent_dim, 5]
        self.conv1 = Conv1d(1, conv1d_channels[0], conv1d_kws[0],
                            conv1d_kws[0])
        self.maxpool1d = MaxPool1d(2, 2)
        self.conv2 = Conv1d(conv1d_channels[0], conv1d_channels[1],
                            conv1d_kws[1], 1)
        dense_dim = int((self.k - 2) / 2 + 1)
        dense_dim = (dense_dim - conv1d_kws[1] + 1) * conv1d_channels[1]
        self.mlp = MLP([dense_dim, 128, 1], dropout=0.5, norm=None)

    def forward(self, x, edge_index, batch):
        xs = [x]
        for conv in self.convs:
            xs += [conv(xs[-1], edge_index).tanh()]
        x = torch.cat(xs[1:], dim=-1)

        # Global pooling.
        x = global_sort_pool(x, batch, self.k)
        x = x.unsqueeze(1)  # [num_graphs, 1, k * hidden]
        x = self.conv1(x).relu()
        x = self.maxpool1d(x)
        x = self.conv2(x).relu()
        x = x.view(x.size(0), -1)  # [num_graphs, dense_dim]

        return self.mlp(x)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCNN(hidden_channels=32, num_layers=3).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = BCEWithLogitsLoss()


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out.view(-1), data.y.to(torch.float))
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs

    return total_loss / len(train_dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    y_pred, y_true = [], []
    for data in loader:
        data = data.to(device)
        logits = model(data.x, data.edge_index, data.batch)
        y_pred.append(logits.view(-1).cpu())
        y_true.append(data.y.view(-1).cpu().to(torch.float))

    return roc_auc_score(torch.cat(y_true), torch.cat(y_pred))


best_val_auc = test_auc = 0
for epoch in range(1, 51):
    loss = train()
    val_auc = test(val_loader)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        test_auc = test(test_loader)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

NameError: name '__file__' is not defined